In [1]:
''''
Taken from: 
https://mtpatter.github.io/bilao/notebooks/html/01-spark-struct-stream-kafka.html

note that the jar file 
org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1
Is aligned to teh current spark version 3.0.1
'''

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'

from ast import literal_eval

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SSKafka") \
    .getOrCreate()
    
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
dsraw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "topic-test") \
  .option("startingOffsets", "earliest") \
  .load()


rawQuery = dsraw \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .start()

raw = spark.sql("select * from qraw")
raw.show() 

'''
rdd_raw = raw.rdd.map(lambda alert: literal_eval(raw['value']))
df = rdd_raw.toDF()

df.write.format("org.elasticsearch.spark.sql").option("es.resource", "rail_all").option("es.nodes", "elasticsearch:9200").save()


# Write toe elastic search

es_write_conf = {
        "es.nodes" : "localhost",
        "es.port" : "9200",
        "es.resource" : 'walker/apache',
        "es.input.json": "yes",
        "es.mapping.id": "doc_id"
    }

rdd2 = rdd.map(parse)

rdd3 = rdd2.map(addID    
       
rdd3.saveAsNewAPIHadoopFile(
        path='-',
        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",       
        keyClass="org.apache.hadoop.io.NullWritable",
        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
        conf=es_write_conf)

rdd3 = rdd2.map(addID

'''

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



'\nrdd_raw = raw.rdd.map(lambda alert: literal_eval(raw[\'value\']))\ndf = rdd_raw.toDF()\n\ndf.write.format("org.elasticsearch.spark.sql").option("es.resource", "rail_all").option("es.nodes", "elasticsearch:9200").save()\n\n\n# Write toe elastic search\n\nes_write_conf = {\n        "es.nodes" : "localhost",\n        "es.port" : "9200",\n        "es.resource" : \'walker/apache\',\n        "es.input.json": "yes",\n        "es.mapping.id": "doc_id"\n    }\n\nrdd2 = rdd.map(parse)\n\nrdd3 = rdd2.map(addID    \n       \nrdd3.saveAsNewAPIHadoopFile(\n        path=\'-\',\n        outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",       \n        keyClass="org.apache.hadoop.io.NullWritable",\n        valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",\n        conf=es_write_conf)\n\nrdd3 = rdd2.map(addID\n\n'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'


KAFKA_TOPIC_NAME_CONS = "topic-test"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'kafka:9092'

if __name__ == "__main__":
    print("PySpark Structured Streaming with Kafka Demo Application Started ...")

    spark = SparkSession \
        .builder \
        .appName("PySpark Structured Streaming with Kafka Demo") \
        .master("local[*]") \
        .getOrCreate()

    spark.sparkContext.setLogLevel("DEBUG")

    # Construct a streaming DataFrame that reads from testtopic
    transaction_detail_df = spark \
        .readStream \
        .format("kafka") \
        .queryName("demo")\
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
        .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
        .option("startingOffsets", "latest") \
        .load()

    print("Printing Schema of transaction_detail_df: ")
    transaction_detail_df.printSchema()

    transaction_detail_df1 = transaction_detail_df.selectExpr("CAST(value AS STRING)", "timestamp")

    raw = spark.sql("select * from qraw")
    raw.show() 
    
    '''
    
    # Define a schema for the transaction_detail data
    transaction_detail_schema = StructType() \
        .add("transaction_id", StringType()) \
        .add("transaction_card_type", StringType()) \
        .add("transaction_amount", StringType()) \
        .add("transaction_datetime", StringType())

    transaction_detail_df2 = transaction_detail_df1\
        .select(from_json(col("value"), transaction_detail_schema).alias("transaction_detail"), "timestamp")

    transaction_detail_df3 = transaction_detail_df2.select("transaction_detail.*", "timestamp")

    # Simple aggregate - find total_transaction_amount by grouping transaction_card_type
    transaction_detail_df4 = transaction_detail_df3.groupBy("transaction_card_type")\
        .agg({'transaction_amount': 'sum'}).select("transaction_card_type", \
        col("sum(transaction_amount)").alias("total_transaction_amount"))

    print("Printing Schema of transaction_detail_df4: ")
    transaction_detail_df4.printSchema()

    transaction_detail_df5 = transaction_detail_df4.withColumn("key", lit(100))\
                                                    .withColumn("value", concat(lit("{'transaction_card_type': '"), \
                                                    col("transaction_card_type"), lit("', 'total_transaction_amount: '"), \
                                                    col("total_transaction_amount").cast("string"), lit("'}")))

    print("Printing Schema of transaction_detail_df5: ")
    transaction_detail_df5.printSchema()
    '''
    # Write final result into console for debugging purpose
    trans_detail_write_stream = transaction_detail_df5 \
        .writeStream \
        .trigger(processingTime='1 seconds') \
        .outputMode("update") \
        .option("truncate", "false")\
        .format("console") \
        .start()

    '''
    print("PySpark Structured Streaming with Kafka Demo Application Completed.")

PySpark Structured Streaming with Kafka Demo Application Started ...
Printing Schema of transaction_detail_df: 
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

Printing Schema of transaction_detail_df4: 
root
 |-- transaction_card_type: string (nullable = true)
 |-- total_transaction_amount: double (nullable = true)

Printing Schema of transaction_detail_df5: 
root
 |-- transaction_card_type: string (nullable = true)
 |-- total_transaction_amount: double (nullable = true)
 |-- key: integer (nullable = false)
 |-- value: string (nullable = true)

PySpark Structured Streaming with Kafka Demo Application Completed.


In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'


from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


KAFKA_TOPIC_NAME_CONS = "topic-test"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'kafka:9092'


# Set Elasticsearch config
es_hostname='localhost'
es_portno='9200'
es_doc_type_name='rail_all'

if __name__ == "__main__":
    print("Spark job starting ...")

    # Create a spark session
    spark = SparkSession \
        .builder \
        .appName("PySpark Structured Streaming with Kafka Demo") \
        .master("local[*]") \
        .getOrCreate()
    
    spark.sparkContext.setLogLevel("DEBUG")


    # Consume from topic
    df_message = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
        .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
        .option("startingOffsets","earliest") \
        .load()

    print("Printing Schema of df_message: ")
    df_message.printSchema()
    
    # convert the data into string
    df_message_string = df_message.selectExpr("CAST(value AS STRING) as value")
    
    
     # Start running the query that prints the running counts to the console
    query = df_message_string \
        .writeStream \
        .format("console") \
        .start()

    query.awaitTermination()
    
    print("Finish ")



Spark job starting ...
Printing Schema of df_message: 
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

